In [26]:
import pandas as pd
import numpy as np
import csv
import os 
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import math 
from dateutil import parser
from joblib import load,dump
 
all_data= []
all_data= pd.read_csv('C:\\Users\\Victor\\Projet_Tennis\\Data\\all_data_co.csv', error_bad_lines=False, sep =',')

data_age = pd.read_csv('C:\\Users\\Victor\\Projet_Tennis\\Data\\data_age.csv', error_bad_lines=False, sep =';')
data_to_invent = pd.read_csv('C:\\Users\\Victor\\Projet_Tennis\\Data\\data_test_co.csv', error_bad_lines=False, sep =',')

In [18]:
def coeffTime(t,a):
    return math.exp(-a*t)


def coeffSurface(s,surface,b):
    if s==surface:
        return 1
    return b




def datematch(x,df,date):
    q=df.iloc[x]['date_tourney'].rsplit('-')
    v=date.rsplit('-')
    S=float(v[1])+12*(float(v[0])-float(q[0]))-float(q[1])
    return max(S,0)

def nb_gamesH2H(player_name_1,player_name_2,all_data):
    return len(all_data[((all_data.player_1_name == player_name_1)&(all_data.player_2_name == player_name_2))| ((all_data.player_2_name == player_name_1)& (all_data.player_1_name == player_name_2))])
def nb_victoryH2H(player_name_1,player_name_2, all_data):
       return len(all_data[((all_data.player_1_name == player_name_1) & (all_data.player_1_win ==1) &(all_data.player_2_name == player_name_2))|((all_data.player_1_name == player_name_2)&(all_data.player_1_win == 0)&(all_data.player_2_name == player_name_1))])
        

def prct_victoryH2H(player_name_1,player_name_2,all_data):
    if nb_gamesH2H(player_name_2,player_name_1,all_data) > 0:
        return nb_victoryH2H(player_name_1,player_name_2,all_data)/nb_gamesH2H(player_name_1,player_name_2,all_data) -  nb_victoryH2H(player_name_2,player_name_1,all_data)/nb_gamesH2H(player_name_2,player_name_1,all_data)
    else:
        return 0


def nb_games(player_name,all_data):
    return len(all_data[((all_data.player_1_name == player_name)| (all_data.player_2_name == player_name))])
def nb_victory(player_name, all_data):
       return len(all_data[((all_data.player_1_name == player_name) & (all_data.player_1_win ==1))])
        
    
def prct_victory(player_name,all_data):
    if nb_games(player_name,all_data) > 0:
        return nb_victory(player_name,all_data)/nb_games(player_name,all_data)
    else:
        return 0
def age(player_1_name,player_2_name,data_age):
    
        date_1 = parser.parse(data_age['DATE_P'][(data_age.NAME_P == player_1_name)].iloc[0])
        date_1 = date_1.replace(tzinfo=None)
        date_2= parser.parse(data_age['DATE_P'][(data_age.NAME_P == player_2_name)].iloc[0])
        date_2= date_2.replace(tzinfo = None)
        a = date_2 - date_1
                  
        return ((a.days)/365)/5.59

In [28]:
def common_opponents(player_1_name,player_2_name,date,data_age,all_data,a,s,b):
    
    ''' Takes in arguments 2 names, a date, a surface (s) ( a and b are hyperparameters)
    Return a line of a dataframe exactly like in all_data_ml for example (every database which ends by _ml)
    First, a set of common opponents of the two players is found (the players which both players have played
    against). Next, we take each common opponent in turn, and find the average performance of both
    players against the common opponent. Finally, we average the performance values for each player across
    all common opponents. In this way, performance estimates for an upcoming match are based on the
    same set of opponents for both players.
    If there are no common opponents, return the mean of the performances of the player with a disadvantage in uncertainty
    '''
    all_data = all_data.drop(all_data[((all_data['player_1_name'] == player_1_name) & (all_data['player_2_name'] ==player_2_name))|((all_data['player_2_name'] ==player_1_name) & (all_data['player_1_name'] ==player_2_name))].index, 
               inplace=False)
    A = all_data[((all_data.player_1_name == player_1_name)|(all_data.player_2_name ==player_1_name))]
    B = all_data[((all_data.player_2_name == player_2_name)|(all_data.player_1_name ==player_2_name))]
    st = np.array([0.11065803769051157,0.18605775919377326,0.14848870568594869,0.3749499037947495,0.1485370107701822,
               0.14846565617960913,0.29700440902994363,0.06561937849862796,0.053872449212457665,0.16164097874108146, 1])
 
    #On sépare en deux la base de données, d'un coté les matchs des adversaires du joueur_1 et de l'autre celui du joueur_2
    
    C_a =pd.concat([B.player_1_name,B.player_2_name]).unique()
    C_b =pd.concat([A.player_1_name,A.player_2_name]).unique()
    C_a = C_a.tolist()
    C_b = C_b.tolist()
    A = A.reset_index()
    B = B.reset_index()
    
    
    X = list(set(C_a)&set(C_b))
    if player_1_name in X:
        X.remove(player_1_name)
    if player_2_name in X:    
        X.remove(player_2_name)
    if X != []:
        lt_A = []
        for i in range(len(A)):
            if ((A.player_1_name.loc[i] not in X) and (A.player_2_name.loc[i] not in X)):
                lt_A.append(i)
        lt_B = []
        for i in range(len(B)):
            if ((B.player_1_name.loc[i] not in X) and (B.player_2_name.loc[i] not in X)):
                lt_B.append(i)
    
    
        A = A.drop(lt_A, axis = 0 )
        B = B.drop(lt_B,axis = 0)
        del A['index']
        del A['Unnamed: 0']
        del B['index']
        del B['Unnamed: 0']
    
        A_1 = A[A.player_1_name == player_1_name]
        A_2 = A[A.player_2_name == player_1_name]
        B_1 = B[B.player_1_name == player_2_name]
        B_2 = B[B.player_2_name == player_2_name]
    
        for c in A_1.columns:
            if 'player_2' in c:
                del A_1[c]
        for c in A_2.columns:
            if 'player_1' in c:
                del A_2[c]
    
        for c in B_1.columns:
            if 'player_2' in c:
                del B_1[c]
        for c in B_2.columns:
            if 'player_1' in c:
                del B_2[c]
    

        columns_to_change = []
        for c in A_2.columns:
            if c.startswith('player_2'):
                columns_to_change.append(c)
    
    
    
        rename_mapping = {}
        for c in columns_to_change:
            if c.startswith("player_2"):
                 rename_mapping[c] = c.replace("player_2", "player_1")
        A_2.rename(columns=rename_mapping, inplace=True)
        del A_1['player_1_win']
        B_2.rename(columns=rename_mapping, inplace=True)
        del B_1['player_1_win']
    
        A_3= A_1.append(A_2)
        B_3= B_1.append(B_2)
    
        lt_coef_time_A =[]
        for i in (range(0,len(A_3))):
            lt_coef_time_A.append(coeffTime(datematch(i,A_3,date),a))
    
        A_3.insert(0, "coef_time_match", lt_coef_time_A, allow_duplicates = False)
    
        lt_coef_time_B =[]
        for i in (range(0,len(B_3))):
            lt_coef_time_B.append(coeffTime(datematch(i,B_3,date),a))
    
        B_3.insert(0, "coef_time_match", lt_coef_time_B, allow_duplicates = False)
    
        lt_coef_surface_a =[]
        for i in (range(0,len(A_3))):
            lt_coef_surface_a.append(coeffSurface(s,all_data['surface'].iloc[i],b))

        A_3.insert(1, "coef_surface_match", lt_coef_surface_a, allow_duplicates = False)
    
        lt_coef_surface_b =[]
        for i in (range(0,len(B_3))):
            lt_coef_surface_b.append(coeffSurface(s,all_data['surface'].iloc[i],b))

        B_3.insert(1, "coef_surface_match", lt_coef_surface_b, allow_duplicates = False)
        
        A_3['coef'] = A_3['coef_surface_match'].multiply(A_3['coef_time_match'],axis = 'rows')   
        B_3['coef'] = B_3['coef_surface_match'].multiply(B_3['coef_time_match'],axis = 'rows')    
    
        A_4 = A_3.iloc[:,8:]

        A_4 = A_4.multiply(A_3['coef'], axis='rows', level=None, fill_value=None)
        
        B_4 = B_3.iloc[:,8:]

        B_4 = B_4.multiply(B_3['coef'], axis='rows', level=None, fill_value=None)
    
    
    
        A_4 = A_4[['player_1_fs_prct','player_1_fs_pt_prct','player_1_ss_prct',
                   'player_1_def_prct','player_1_bp_prct','player_1_wsp','player_1_serveadv','player_1_aces_pcrt',
                   'player_1_df_pcrt', 'player_1_completeness','coef']]
        B_4 = B_4[['player_1_fs_prct','player_1_fs_pt_prct','player_1_ss_prct',
                   'player_1_def_prct','player_1_bp_prct','player_1_wsp','player_1_serveadv','player_1_aces_pcrt',
                   'player_1_df_pcrt', 'player_1_completeness','coef']]
        lt_co = []
        lt_co = A_4.mean()/st - B_4.mean()/st 
  
        s1 = pd.Series([s,prct_victory(player_1_name,all_data)-prct_victory(player_2_name,all_data),1/(0.001 + B_3['coef'].sum()*A_3['coef'].sum()),
                   age(player_1_name,player_2_name,data_age),player_1_name,player_2_name,prct_victoryH2H(player_1_name,player_2_name,all_data)] 
                   ,index = ['surface','win_ratio_diff','uncertainty','diff_age','name_player_1','name_player_2','H2H'])
    
    
        lt_co = lt_co.append(s1)
        lt_co = lt_co.to_frame().T[['name_player_1','name_player_2','player_1_fs_prct','player_1_fs_pt_prct','player_1_ss_prct',
                   'player_1_def_prct','player_1_bp_prct','player_1_wsp','player_1_serveadv','player_1_aces_pcrt',
                   'player_1_df_pcrt','diff_age', 'player_1_completeness','win_ratio_diff','surface','uncertainty','H2H']]
    
    
        return  lt_co
    else:
        
        del A['index']
        del A['Unnamed: 0']
        del B['index']
        del B['Unnamed: 0']
        A = A[((A.player_1_name != player_1_name)&(A.player_2_name !=player_2_name))|((A.player_1_name != player_2_name)&(A.player_2_name !=player_1_name))]
        B = B[((B.player_1_name != player_1_name)&(B.player_2_name !=player_2_name))|((B.player_1_name != player_2_name)&(B.player_2_name !=player_1_name))]
        A_1 = A[A.player_1_name == player_1_name]
        A_2 = A[A.player_2_name == player_1_name]
        B_1 = B[B.player_1_name == player_2_name]
        B_2 = B[B.player_2_name == player_2_name]
    
        for c in A_1.columns:
            if 'player_2' in c:
                del A_1[c]
        for c in A_2.columns:
            if 'player_1' in c:
                del A_2[c]
    
        for c in B_1.columns:
            if 'player_2' in c:
                del B_1[c]
        for c in B_2.columns:
            if 'player_1' in c:
                del B_2[c]
    

        columns_to_change = []
        for c in A_2.columns:
            if c.startswith('player_2'):
                columns_to_change.append(c)
    
    
    
        rename_mapping = {}
        for c in columns_to_change:
            if c.startswith("player_2"):
                 rename_mapping[c] = c.replace("player_2", "player_1")
        A_2.rename(columns=rename_mapping, inplace=True)
        del A_1['player_1_win']
        B_2.rename(columns=rename_mapping, inplace=True)
        del B_1['player_1_win']
    
        A_3= A_1.append(A_2)
        B_3= B_1.append(B_2)
    
        lt_coef_time_A =[]
        for i in (range(0,len(A_3))):
            lt_coef_time_A.append(coeffTime(datematch(i,A_3,date),a))
    
        A_3.insert(0, "coef_time_match", lt_coef_time_A, allow_duplicates = False)
    
        lt_coef_time_B =[]
        for i in (range(0,len(B_3))):
            lt_coef_time_B.append(coeffTime(datematch(i,B_3,date),a))
    
        B_3.insert(0, "coef_time_match", lt_coef_time_B, allow_duplicates = False)
    
        lt_coef_surface_a =[]
        for i in (range(0,len(A_3))):
            lt_coef_surface_a.append(coeffSurface(s,all_data['surface'].iloc[i],b))

        A_3.insert(1, "coef_surface_match", lt_coef_surface_a, allow_duplicates = False)
    
        lt_coef_surface_b =[]
        for i in (range(0,len(B_3))):
            lt_coef_surface_b.append(coeffSurface(s,all_data['surface'].iloc[i],b))

        B_3.insert(1, "coef_surface_match", lt_coef_surface_b, allow_duplicates = False)
        
        A_3['coef'] = A_3['coef_surface_match'].multiply(A_3['coef_time_match'],axis = 'rows')   
        B_3['coef'] = B_3['coef_surface_match'].multiply(B_3['coef_time_match'],axis = 'rows')    
    
        A_4 = A_3.iloc[:,8:]

        A_4 = A_4.multiply(A_3['coef'], axis='rows', level=None, fill_value=None)
        
        B_4 = B_3.iloc[:,8:]

        B_4 = B_4.multiply(B_3['coef'], axis='rows', level=None, fill_value=None)
    
    
    
        A_4 = A_4[['player_1_fs_prct','player_1_fs_pt_prct','player_1_ss_prct',
                   'player_1_def_prct','player_1_bp_prct','player_1_wsp','player_1_serveadv','player_1_aces_pcrt',
                   'player_1_df_pcrt', 'player_1_completeness','coef']]
        B_4 = B_4[['player_1_fs_prct','player_1_fs_pt_prct','player_1_ss_prct',
                   'player_1_def_prct','player_1_bp_prct','player_1_wsp','player_1_serveadv','player_1_aces_pcrt',
                   'player_1_df_pcrt', 'player_1_completeness','coef']]
        lt_co = []
        lt_co = A_4.mean()/st - B_4.mean()/st 
  
        s1 = pd.Series([s,prct_victory(player_1_name,all_data)-prct_victory(player_2_name,all_data),1/(0.001 + B_3['coef'].sum()*A_3['coef'].sum())+ 0.9,
                   age(player_1_name,player_2_name,data_age),player_1_name,player_2_name,prct_victoryH2H(player_1_name,player_2_name,all_data)] 
                   ,index = ['surface','win_ratio_diff','uncertainty','diff_age','name_player_1','name_player_2','H2H'])
    
    
        lt_co = lt_co.append(s1)
        lt_co = lt_co.to_frame().T[['name_player_1','name_player_2','player_1_fs_prct','player_1_fs_pt_prct','player_1_ss_prct',
                   'player_1_def_prct','player_1_bp_prct','player_1_wsp','player_1_serveadv','player_1_aces_pcrt',
                   'player_1_df_pcrt','diff_age', 'player_1_completeness','win_ratio_diff','surface','uncertainty','H2H']]
    
    
        return  lt_co
        



In [29]:
common_opponents('Rafael Nadal','Roger Federer','2021-06',data_age,all_data,0.22,1,1/2)

name_player_1  name_player_2 player_1_fs_prct player_1_fs_pt_prct  \
0  Rafael Nadal  Roger Federer       0.00210417          0.00143148   

  player_1_ss_prct player_1_def_prct player_1_bp_prct player_1_wsp  \
0       0.00143653       0.000484318       0.00126945   0.00165781   

  player_1_serveadv player_1_aces_pcrt player_1_df_pcrt  diff_age  \
0       0.000212983        0.000210821      0.000167906 -0.818977   

  player_1_completeness win_ratio_diff surface uncertainty H2H  
0           0.000654251     -0.0199823       1     9.20906   0

In [30]:

def prediction(lt_match,data_age,all_data):
    reg_loaded = load('regression_model_saved.joblib')
    svm_loaded= load('svc_model_saved.joblib')
    s = pd.DataFrame([])
    lt_player_1_name,lt_player_2_name,lt_date,lt_odd_p1,lt_odd_p2,lt_surface =[], [],[],[],[],[]  
    
    for i in range(len(lt_match)):
    
        lt_player_1_name.append(lt_match[i][0])
        lt_player_2_name.append(lt_match[i][1])
        lt_date.append(lt_match[i][2])
        lt_surface.append(lt_match[i][3])
        lt_odd_p1.append(lt_match[i][4])
        lt_odd_p2.append(lt_match[i][5])
        lt_diff_odds = [b_i/4.4 - a_i/4.4 for a_i, b_i in zip(lt_odd_p1, lt_odd_p2)]
    player_1_name = pd.DataFrame(lt_player_1_name,columns=['player_1_name'])
    data_to_predict = player_1_name.join([pd.DataFrame(lt_player_2_name,columns=['player_2_name']),pd.DataFrame(lt_date,columns=['date']),
                                         pd.DataFrame(lt_surface,columns =['surface']),pd.DataFrame(lt_diff_odds, columns =["diff_odds"])])

    for j in range(len(lt_match)):
            s= s.append(common_opponents(data_to_predict['player_1_name'].iloc[j], data_to_predict['player_2_name'].iloc[j],data_to_predict['date'].iloc[j],data_age,all_data,0.22,data_to_predict['surface'].iloc[j],1/2))
    
    s_predict = data_to_predict.join(s.reset_index(),lsuffix='_caller', rsuffix='_other')
    s_model = s_predict[['diff_odds','player_1_fs_prct','player_1_fs_pt_prct','player_1_ss_prct','player_1_def_prct',
     'player_1_bp_prct','player_1_wsp','player_1_serveadv','player_1_aces_pcrt','player_1_df_pcrt','diff_age','player_1_completeness',
      'surface_caller','win_ratio_diff','H2H']]


    lt = []
    
    for k in range(len(lt_match)):
        lt.append((reg_loaded.predict_proba(s_model)[k][1],lt_player_1_name[k]))
    return lt , s_predict['uncertainty']


In [34]:
prediction([('Rafael Nadal','Novak Djokovic','2021-06',1,1.2,3.5)],data_age,all_data)

C:\Users\Victor\anaconda3\lib\site-packages\pandas\core\frame.py:4125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


([(0.540580094861148, 'Rafael Nadal')],
 0    0.82919
 Name: uncertainty, dtype: object)